# Board Game Similarity Search

### Use descriptions of Board Games from [Board Game Geek](https://boardgamegeek.com/browse/boardgame) website to find similar board games

In [1]:
import pandas as pd
import numpy as np
import similarity_search
from IPython.display import HTML

In [2]:
def pretty_show(game_ranks):
    img_urls = bg_df.loc[game_ranks, 'thumbnail'].to_list()
    game_urls = bg_df.loc[game_ranks, 'url'].to_list()
    return img_urls, game_urls

def get_html_code(img_urls, game_urls):
    html_code ="""
<style>
    img {
        float: left;
        padding: 8px;
    }
</style>
    """
    for img_url, game_url in zip(img_urls, game_urls):
        html_code += """
<a href="%s"><img src="%s" style="width: 300px; height: 300px;"></a>
        """ % (game_url, img_url)

    return html_code

In [3]:
# top 2000 board games
bg_df = pd.read_csv("bgg_2000.csv")

In [7]:
# enter rank - 1 for the selected game
selected_game_rank = 285
results = similarity_search.get_similar(selected_game_rank)[:9]
game_names = bg_df.loc[results, 'name'].to_list()
print("Games similar to ->", bg_df.loc[selected_game_rank, 'name'])
print('-----')
print('\n'.join(game_names))

Games similar to -> Century: Spice Road
-----
Québec
The Voyages of Marco Polo
Century: A New World
Century: Eastern Wonders
Century: Golem Edition
Yspahan
Kashgar: Merchants of the Silk Road


In [8]:
img_urls, game_urls = pretty_show([selected_game_rank])
HTML(get_html_code(img_urls, game_urls))

In [9]:
img_urls, game_urls = pretty_show(results)
HTML(get_html_code(img_urls, game_urls))